# Setting backend options

A `RunningManBackend` has three sets of options that can be used to configure circuit execution.  Lets see how to use those:

In [1]:
from qiskit import *
import runningman as rm

In [2]:
provider = rm.RunningManProvider()

In [3]:
backend = provider.backend('ibm_brisbane')

## Retriving execution and suppression options

In [4]:
options = backend.get_execution_options()
options

Execution Options
⏵execution
  default_shots          4096
  experimental           None
  init_qubits            True
  max_execution_time     None
  meas_type              'classified'
  rep_delay              None
⏵environment
  log_level              'WARNING'
  callback               None
  job_tags               None
  private                False
⏵simulator
  noise_model            None
  seed_simulator         None
  coupling_map           None
  basis_gates            None

In [5]:
sup_options = backend.get_suppression_options()
sup_options

Suppression Options
⏵dynamical_decoupling
  enable                       False
  sequence_type                'XY4'
  extra_slack_distribution     'middle'
  scheduling_method            'alap'
⏵twirling
  enable_gates                 False
  enable_measure               False
  num_randomizations           'auto'
  shots_per_randomization      'auto'
  strategy                     'active-accum'

These options are just dictionaries, and can easily be set.  I can modify the returned classes directly and pass them back to the backend:

In [6]:
sup_options['dynamical_decoupling']['enable'] = True
backend.set_suppression_options(**sup_options)
backend.get_suppression_options()

Suppression Options
⏵dynamical_decoupling
  enable                       True
  sequence_type                'XY4'
  extra_slack_distribution     'middle'
  scheduling_method            'alap'
⏵twirling
  enable_gates                 False
  enable_measure               False
  num_randomizations           'auto'
  shots_per_randomization      'auto'
  strategy                     'active-accum'

or I can set them in the setter:

In [8]:
backend.set_suppression_options(dynamical_decoupling={'enable':False})
backend.get_suppression_options()

Suppression Options
⏵dynamical_decoupling
  enable                       False
  sequence_type                'XY4'
  extra_slack_distribution     'middle'
  scheduling_method            'alap'
⏵twirling
  enable_gates                 False
  enable_measure               False
  num_randomizations           'auto'
  shots_per_randomization      'auto'
  strategy                     'active-accum'

## Example

Here we execute three jobs in a `Batch` and show how one can manipulate the number of shots

In [46]:
qc = QuantumCircuit(5)
qc.h(2)
qc.cx(2, 1)
qc.cx(1, 0)
qc.cx(2, 3)
qc.cx(3, 4)
qc.measure_all()

In [47]:
trans_qc = transpile(qc, backend)

In [48]:
jobs = []
backend.set_mode('batch')
# default 4096 shots
jobs.append(backend.run(trans_qc))
# overloaded 1234 shots
jobs.append(backend.run(trans_qc, shots=1234))
# set 4321 shots
backend.set_execution_options(execution={'default_shots':4321})
jobs.append(backend.run(trans_qc))
backend.close_mode()

In [49]:
[sum(job.result().get_counts().values()) for job in jobs]

[4096, 1234, 4321]